<a href="https://colab.research.google.com/github/rushi-k12/Finetuning/blob/main/Finetuning_LLM_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets

In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load dataset
dataset = load_dataset("glue", "mrpc")

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

# Train the model
trainer.train()



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,No log,0.512871


TrainOutput(global_step=230, training_loss=0.5791423631751019, metrics={'train_runtime': 4590.0093, 'train_samples_per_second': 0.799, 'train_steps_per_second': 0.05, 'total_flos': 241272837765120.0, 'train_loss': 0.5791423631751019, 'epoch': 1.0})

In [5]:
pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [1]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes

# Load dataset
data = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

# Enable MLflow autologging
mlflow.sklearn.autolog()

with mlflow.start_run():
    # Train model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Predict and log metrics
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    mlflow.log_metric("mse", mse)

    print(f"Mean Squared Error: {mse}")


2024/06/19 12:58:06 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


Mean Squared Error: 2952.0105887640448


In [4]:
import json

data = [
    {"prompt": "Identify if the clause provides payment terms.", "text": "The contract specifies that payment should be made within 30 days of invoice.", "answer": "yes"},
    {"prompt": "Identify if the clause provides termination conditions.", "text": "Either party may terminate this agreement with 60 days' notice.", "answer": "yes"},
    {"prompt": "Identify if the clause provides confidentiality obligations.", "text": "The parties agree to keep all terms of this agreement confidential.", "answer": "yes"},
    {"prompt": "Identify if the clause provides dispute resolution mechanisms.", "text": "Any disputes arising from this contract will be resolved through arbitration.", "answer": "yes"},
    {"prompt": "Identify if the clause provides a governing law.", "text": "This agreement shall be governed by the laws of the State of New York.", "answer": "yes"},
    {"prompt": "Identify if the clause provides assignment rights.", "text": "The parties may assign their rights under this agreement to third parties.", "answer": "yes"},
    {"prompt": "Identify if the clause provides payment terms.", "text": "The contractor will complete the work in a timely manner.", "answer": "no"},
    {"prompt": "Identify if the clause provides termination conditions.", "text": "The parties agree to collaborate on future projects.", "answer": "no"},
    {"prompt": "Identify if the clause provides confidentiality obligations.", "text": "The contractor will deliver the project in accordance with the specifications.", "answer": "no"},
    {"prompt": "Identify if the clause provides dispute resolution mechanisms.", "text": "The deliverables will be accepted upon successful completion of tests.", "answer": "no"}
]

with open('dataset.jsonl', 'w') as f:
    for item in data:
        f.write(json.dumps(item) + "\n")

print("Dataset saved as 'dataset.jsonl'")


Dataset saved as 'dataset.jsonl'


In [5]:
from datasets import load_dataset

dataset = load_dataset('json', data_files='dataset.jsonl')
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'text', 'answer'],
        num_rows: 10
    })
})
